In [8]:
from csv import writer
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
import requests
from bs4 import BeautifulSoup
df = pd.read_excel('Input.xlsx')
df1 = df.reset_index()['URL']#selecting only the URL from the excel and saving it in df1
df2 = df.reset_index()['URL_ID']
#df6 = []
#for x in range(0,149):
 #   df6.append(df1[x])
#df6.reverse()
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})
def removal(fname, text):
    file = open(fname,"r")
    f = file.read().split()
    final = ""
    for word in text:
        for g in f:
            if word == g:
                word = word.replace(g,'')
        #print(word)
        final = final+word+" "
    final_list = final.split()
    return final_list
def score(fname,texts):
    file = open(fname,"r")
    f = file.read().split()
    count = 0
    for i in texts:
        for j in f:
            if i == j :
                count = count +1
    file.flush()
    file.close()
    return count        
with open('Output Data Structure.csv','w',encoding='utf-8',newline='') as fn:
            thewriter = writer(fn)
            header = ['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']
            thewriter.writerow(header)
            url_id = 0
            for url in df6:    
                    url_id = url_id+1
                    print(url_id)
                    html = requests.get(url,headers=headers)
                    links = requests.get(url,headers=headers)
                    link = BeautifulSoup(links.content,'html.parser')
                    content = link.find_all('div', class_="detail")
                    titles = link.find_all('h1',class_='hdg1')
                    clean = open("cleaning.txt","r",encoding='utf-8')
                    b = clean.read().split()
                    clean.close()
                    with open('data.txt', 'r+', encoding='utf-8') as f:
                        #print(r['body'], file=f)
                    #file = open("data.txt","w")
                        para = BeautifulSoup(str(content), "lxml").text
                        title = BeautifulSoup(str(titles), "lxml").text
                        l = title.split()
                        #print(l)
                        a = para.split()
                        new = ""
                        for x in l:
                            for y in b:
                                x= x.replace(y,' ')
                            new = new+x+" " 
                        print(new)
                        for x in a:
                            for y in b:
                                x= x.replace(y,' ')
                           #print(x)
                            new = new+x+" "
                        j = new
                        new = new.upper()
                        new = new.strip()
                        t = new.split()
                        text = new.split()
                        list_stopWords = ['StopWords_Auditor.txt','StopWords_Currencies.txt','StopWords_DatesandNumbers.txt','StopWords_Generic.txt','StopWords_GenericLong.txt','StopWords_Geographic.txt','StopWords_Names.txt']
                        for i in list_stopWords:
                            text = removal(i,text)
                        after_cleaning = ""
                        for i in text:
                            after_cleaning = after_cleaning+i+" "
                        #print(after_cleaning)
                        #print(text)
                        #print(len(text))
                        f.write(after_cleaning)
                        #cleaning = '''[]<>/"'-='''  
                        f.flush()
                        f.close()
                    #print(text)
                    positive_score = 0
                    negative_score = 0
                    with open('data.txt', 'r', encoding='utf-8') as f:
                        texts = f.read().lower().split()
                        #print(texts)
                        words_after_cleaning = len(texts)
                        list_data_analysis = ['negative-words.txt','positive-words.txt']
                        score_list = [0,0]
                        c = 0
                        for i in list_data_analysis:
                            number = score(i,texts)
                            score_list[c] = int (number)
                            c = c+1
                        #print(score_list)
                        #print(words_after_cleaning)
                        negative_score = score_list[0]
                        positive_score = score_list[1]
                        #print(positive_score)
                        #print(texts)
                    polarity_score = (score_list[1]-score_list[0])/((score_list[1]+score_list[0])+0.000001)
                    subjectivity_score = (score_list[1]+score_list[0])/ ((words_after_cleaning) + 0.000001)
                    number_of_words = len(t)
                    number_of_sentences = sent_tokenize(new)
                    sentence_count = len(number_of_sentences)
                    print(sentence_count)
                    Average_Sentence_Length = number_of_words/sentence_count
                    complex_words = 0
                    for words in t:
                        length = len(words)
                        if length > 2:
                            complex_words = complex_words+1
                    Percentage_of_Complex_words = complex_words/number_of_words
                    Fog_Index = 0.4 * (Average_Sentence_Length + Percentage_of_Complex_words)
                    AVG_NUMBER_OF_WORDS_PER_SENTENCE = Average_Sentence_Length 
                    stop_words = set(stopwords.words('english')) 
                    s = [x.upper() for x in stop_words]
                    #print(s)
                    new_s = ""
                    for r in t: 
                        for i in s: 
                            if i == r:
                                check = 0
                                break
                            else:
                                check = 1
                        if check == 1:
                            new_s = new_s+r+" "
                    punctuation_list = ['!','.','?',',']
                    new_filter = ""
                    for char in new_s:
                        for i in punctuation_list:
                            if char == i:
                                char = char.replace(i,'')
                        new_filter = new_filter+char
                    word_count = len(new_filter.split())
                    vowel_list = ['A','E','I','O','U']
                    Syllable_Count_Per_Word = 0
                    for word in t:
                        for char in word:
                            for i in vowel_list:
                                if char == i:
                                    Syllable_Count_Per_Word = Syllable_Count_Per_Word + 1
                        if word.endswith("ED") or word.endswith("ES"):
                            Syllable_Count_Per_Word = Syllable_Count_Per_Word - 1
                    Personal_pronoun_count = ['I', 'WE', 'MY','OURS','US']
                    Personal_pronoun = 0
                    for word in t:
                        for i in Personal_pronoun_count:
                            if word == i:
                                Personal_pronoun = Personal_pronoun + 1
                    j_count = 0
                    check_j = j.split()
                    for word in check_j:
                        if word == 'US':
                            j_count = j_count+1
                    Personal_pronoun = Personal_pronoun - j_count
                    blank_count = 0
                    for char in new:
                        if char == ' ':
                            blank_count = blank_count +1
                    total_characters = len(new) - blank_count
                    total_characters
                    Average_Word_Length = total_characters/number_of_words
                    info = [url_id,url,positive_score,negative_score,polarity_score,subjectivity_score,Average_Sentence_Length,Percentage_of_Complex_words,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,complex_words,word_count,Syllable_Count_Per_Word,Personal_pronoun,Average_Word_Length]
                    thewriter.writerow(info)
            


1
 US slams Iran for attack on Salman Rushdie, terms it  despicable, disgusting   
20
2
 FIFA bans India  takes away hosting rights of women s under 17 World Cup  
19
3
 Is dyslexia a gift? Here are some surprising benefits  
38
4
 Britain s RAF flights transporting ammunition from Pakistan to Ukraine?  
21
5
 Laal Singh Chaddha box office day 5 collection  Film might shut shop by next week  earns only ₹46 crore so far  
19
6
 Kapil Sharma shares pic with his daughter, Anayra Sharma, calls her  my little world   
24
7
 Aakanksha Singh  As actors we should readily take up challenges  
20
8
 Shah, Nadda meet Bihar BJP core group days after Nitish moves on to RJD  
8
9
  Pandemic far from over   Delhi L G as capital sees rise in Covid 19 deaths  
14
10
 Horoscope Today  Astrological prediction for August 16, 2022  
83
